In [10]:
import numpy as np
import matplotlib.pyplot as plt
import math as m
from mpl_toolkits.mplot3d import Axes3D

#%matplotlib inline

In [40]:
### Startwerte eingeben

#Höhe in metern
H = 1000.

# Anzahl der Simulationen also maximale Kollisionszahl
sim = 1000

# Photonenanzahl
photonen = 3

# Start für X Y Z 
x, y, z= 0, 0, H


#d=1./4.*np.pi  ### Einstrahlungswinkel zu Beginn zum Lot hin (hier 45° ist 1/4 * pi)
l=0

beta=0.01 # in m ???sollte doch mit ssalbedo abhängig sein ????

ssalbedo=0.97#0.99 #0.95#0.99#0.99 # Je kleiner desto weniger Streuung, Je größer desto mehr Streuung

# Streuparameter
g=0.99#0.85 # 

# Seed für Zufallszahlengenerator
rs = np.random.RandomState(42)


In [41]:
#### Wichtige Funktionen 

## Weglänge
def weg(a):
    l = -1. * np.log(1-rs.rand(1)) / a # 
    #l=-np.log10(1-np.random.normal(0,0.2,1))*a
    return l

## Asymetrie Parameter
def Theta_HG(random,g):
    if g==0:
        Theta=np.arccos(2*random-1);
        return Theta
    else:
        #par=(2. * g * random / (1. - g**2) + 1. / (1. +g ))**2;
        #mu=(1+g**2-1./par)/(2*g);
        #Theta = np.arccos(mu)
        res = (1. + g**2) - ((1.- g**2.)/(1.- g + 2.*g*random))**2
        Theta = np.arccos((1./(2. *g)*res))
        return Theta

# Rotations Matrix
def rotmat(theta_old,phi_old):
    R11 = np.cos(theta_old) * np.cos(phi_old)
    R21 = np.cos(theta_old) * np.sin(phi_old)
    R31 = -np.sin(theta_old)
    R12 = -np.sin(phi_old)
    R22 = np.cos(phi_old)
    R32 = 0
    R13 = np.sin(theta_old) * np.cos(phi_old)
    R23 = np.sin(theta_old) * np.sin(phi_old)
    R33 = np.cos(theta_old)
    A = np.matrix([[R11,R21,R31], [R12,R22,R32], [R13,R23,R33]])
    return np.matrix.transpose(A)
    


In [96]:
fig = plt.figure(figsize=(12,12))
ff = 20
### Nur bei 3d notwendig
ax = fig.add_subplot(111,projection='3d')

ground = 0
sky = 0
abso = 0
wabso = 0
    
for jj in range(photonen):
#############################################
    # Von 0 bis pi/2 Quadrant oben rechts wo die Sonnenstrahlen herkommen
    # NEGATIVE Werte drehen die Richtung der Sonnenstrahlen um. Sie kommen aus Quadranten obenlinks
    
    d=(1./6.)*np.pi # Theta sonne
    dp=0 # Phi Sonne
    
    position = np.zeros([sim+1,3]) 
    position[:] = np.NAN
    position[0,0], position[0,1], position[0,2] = x, y, z

    i = 0

    while i < sim:
        
        #while für was assiert an der wolken obergrenze.... es kann direkt zum Abbruch kommen
        #if (np.random.uniform(0,1,1)>ssalbedo): ### Schleife stoppen wenn photon absorbiert wird
        #    wabso = wabso +1
        #    break
        
        
        #EinPhoton dringt in die Wolke ein ...was passiert?
        l = weg(beta)#*H
        #print ('weg: ',l)
        
        # if l > H/cos(theta): tdir = tdir+1 und ground = ground +1 break
        
        # Neue Position
        position[i+1,0] = position[i,0]+np.sin(d)*np.cos(dp)*l # X-position
        position[i+1,1] = position[i,1]+np.sin(d)*np.sin(dp)*l # Y-Position
        position[i+1,2] = position[i,2]-np.cos(d)*l # Z-Position
        
        # Liegt die Neue Position innerhalb der Wolke?
        if (position[i+1,2]<=0):
            ground = ground + 1
            break ### Schleife stoppen wenn photon am Boden ankommt 
            #pass

        if (position[i+1,2]>1.*H):
            sky = sky + 1
            break ### Schleife Stoppen wenn photon wolkenoberkante überschreitet
            #pass

        # Absorbtion mit Wahrscheinlichkeit 1-ssalbedo
        if (np.random.uniform(0,1,1)>ssalbedo): ### Schleife stoppen wenn photon absorbiert wird
            abso = abso +1
            break
            #pass
        else:
            # Rotations Matrix mit ALTEN THETA und PHI
            M = rotmat(d,dp)
            
            # Scatterevent: NEUEN THETA und PHI berechnen 
            d = Theta_HG(rs.rand(1),g)#*np.pi
            dp = np.random.uniform(0,1,1)*np.pi*2
            
            # Neue Richtung mit NEUEN THETA und PHI
            xd_neu = np.sin(d)*np.cos(dp) # X-position
            yd_neu = np.sin(d)*np.sin(dp) # Y-Position
            zd_neu = np.cos(d) # Z-Position
        

            # Matrixmultipliaktion auf neue Position mit M aus alten THETA und PHI
            # Position = position * M
            C = np.dot(M,np.array([xd_neu, yd_neu, zd_neu]))
            
            print (C)
            #print (position[i+1,:])
            #position[i+1,0] =  C[0,0]# X-position
            #position[i+1,1] =  C[0,1]# Y-Position
            #position[i+1,2] =  C[0,2]# Z-Position
            
            # re-transforme to polar
            d = m.acos(C[2])
            dp =m.atan2(C[0],C[1])
            
            #B = np.array([[position[i,0]+np.sin(d)*np.cos(dp)*l],
            #              [position[i,1]+np.sin(d)*np.sin(dp)*l],
            #              [position[i,2]-np.cos(d)*l ]])
            #C = np.dot(np.matrix.transpose(A),B)
            # 
            #position[i+1,0] = C[0] 
            #position[i+1,1] = C[1]
            #position[i+1,2] = C[2]
            
            
        i = i+1
            #d = np.random.normal(0,0.2,1000)*np.pi+d

            
            
            
            
        ### Plot der Trajektorie in 3D
        ax.scatter(position[0,0],position[0,1],position[0,2], s=5, color='red')
        ax.plot(position[:,0],position[:,1],position[:,2], linestyle='-.', color='gray')
        ax.set_xlim(-500,500)
        ax.set_ylim(-500,500)
        ax.set_zlim(0,1000)
        ax.set_xlabel('X Label')
        ax.set_ylabel('Y Label')
        ax.set_zlabel('Z Label')


        ### Plot der Trajektorie in 2D
        #plt.plot(position[:,0],position[:,2], color='black',linewidth=0.5, zorder=1)
        #plt.scatter(position[0,0],position[0,2],s=100, color='red',zorder=2)
        #plt.subplot(1,2,2)
        #plt.plot(position[:,1],position[:,2], color='black',linewidth=0.5, zorder=1)
        #plt.scatter(position[0,1],position[0,2],s=100, color='red',zorder=2)
######################################################


#plt.axis('equal')
#plt.ylim(900,1000)
#plt.xlim(-1000, 1000)
plt.grid()
plt.suptitle('Am Oberrand reflektiert: '+str(sky)+' \n Am Unterrand angekommen: '+str(ground)+
         '\n Absorbiert: ' + str(abso)+ ' von ' + str(photonen) )
plt.title(r'$\omega$= '+str(ssalbedo) + r', g= ' + str(g) + r', $\beta$ = '+ str(beta)+
          r', $\tau$ = '+str(H*beta))
plt.ylabel('Höhe in m')
plt.xlabel('X in m')
plt.savefig('/home/velibor/Bilder/met733/MC3.png')
plt.close()


print (g,beta,photonen, ground,sky,abso)     

[[ 0.50515803]
 [-0.01066998]
 [ 0.8629609 ]]
[[ 0.56417616]
 [-0.39063172]
 [ 0.72740093]]
[[-0.39461887]
 [ 0.57008197]
 [ 0.72061258]]
[[ 0.57185871]
 [-0.39541121]
 [ 0.71876811]]
[[-0.39304499]
 [ 0.57091493]
 [ 0.72081328]]
[[ 0.58186028]
 [-0.36393332]
 [ 0.72731778]]
[[-0.36769863]
 [ 0.57554741]
 [ 0.73044021]]
[[ 0.41147595]
 [-0.39607371]
 [ 0.82086123]]
[[-0.39417337]
 [ 0.41384045]
 [ 0.82058726]]
[[ 0.43195694]
 [-0.39555481]
 [ 0.81052427]]
[[-0.36342371]
 [ 0.4109327 ]
 [ 0.8360966 ]]
[[ 0.42841588]
 [-0.34559271]
 [ 0.83488054]]
[[-0.33963484]
 [ 0.42454639]
 [ 0.8392905 ]]
[[ 0.4308149 ]
 [-0.33847495]
 [ 0.83656036]]
[[-0.36988599]
 [ 0.45217262]
 [ 0.81161831]]
[[ 0.45947087]
 [-0.38345324]
 [ 0.8011555 ]]
[[ 0.491507  ]
 [-0.00472125]
 [ 0.87086083]]
[[-0.0320278 ]
 [ 0.50586594]
 [ 0.86201732]]
[[ 0.51158377]
 [-0.02236117]
 [ 0.85894239]]
[[-0.01812906]
 [ 0.4860166 ]
 [ 0.87376152]]
[[ 0.47471832]
 [ 0.00104953]
 [ 0.88013716]]
[[-0.01217354]
 [ 0.43318582]
 [ 0

In [71]:
C=np.dot(M,position[i+1,:])
print (position[i+1,:])
    

[ 135.3262166   375.44262706  466.70197688]


In [77]:
C[0]



matrix([[ 0.36835661]])

In [110]:
# A = np.matrix([[1,2,3], [2,2,2], [1,23,3]])
print (A)
B = ([[1],[2],[3]])

[[ 1  2  3]
 [ 2  2  2]
 [ 1 23  3]]


In [44]:
np.dot(np.matrix.transpose(A),B)
C = np.dot(A,B)


In [48]:
C[2]

matrix([[56]])